## Import libraries

In [1]:
import pandas as pd
import requests
import tabula

## Import Consumer Expenditure Surveys, U.S. Bureau of Labor Statistics, 2019-2020 (Western US)

In [2]:
#using tabula to parse pdf
dfs = tabula.read_pdf("https://www.bls.gov/cex/tables/cross-tab/mean/cu-region-by-income-west-2020.pdf", pages = "all")

## Find Income + Total Expenditures of Income Groups
- Low-income group is population of households that earns between $15,000 - $30,000.
- High-income group is population of households that earns between $100,000 - $149,999.

In [3]:
#select columns of income groups under analysis
dfSES = dfs[0][['Unnamed: 0','$15,000','$100,000']]
dfSES.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

dfIncomeExpense = dfSES.loc[[9,49]]
dfIncomeExpense['lowIncomeExpense'] = dfIncomeExpense['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfIncomeExpense['highIncomeExpense'] = dfIncomeExpense['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)

dfIncomeExpense = dfIncomeExpense.T
dfIncomeExpense.columns = ['income','expenses']
dfIncomeExpense = dfIncomeExpense[1:]
dfIncomeExpense.index = ["lowIncome","highIncome"]
dfIncomeExpense


,income,expenses
lowIncome,22248,36319
highIncome,121184,87629


## Find Expenditure Categories From the Consumer Expenditure Surveys
- Pick out major expenditure categories that match the BLS' Consumer Price Index estimates and calculate how much each income group spent on the various categories. This is required in order to estimate their inflationary cost in dollar amounts.

In [4]:
dfExpense = pd.concat(dfs).reset_index(drop=True)[['Unnamed: 0','$15,000','$100,000']]
dfExpense.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

#These rows correspond to major category groups that match up to BLS' inflation categories.
dfExpenseToMatch = dfExpense.loc[[51,52,83,85,88,96,105,108,112,120,131,145,151,169,165,167,171,173,175,177]]
dfExpenseToMatch['lowIncomeExpense'] = dfExpenseToMatch['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['highIncomeExpense'] = dfExpenseToMatch['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['category'] = dfExpenseToMatch['category'].str.replace(".","",regex=False).str.rstrip()
dfExpenseToMatch = dfExpenseToMatch.reset_index(drop=True)

#rename and group categories
dfExpenseToMatch = dfExpenseToMatch.replace("Household operations","Household furnishings and operations",regex=True) \
                                   .replace("Housekeeping supplies","Household furnishings and operations",regex=True) \
                                   .replace("Household furnishings and equipment","Household furnishings and operations",regex=True) \
                                   .replace("Reading","Other goods and services",regex=True) \
                                   .replace("Tobacco products and smoking supplies","Other goods and services",regex=True) \
                                   .replace("Miscellaneous","Other goods and services",regex=True) \
                                   .replace("Cash contributions","Other goods and services",regex=True) \
                                   .replace("Personal insurance and pensions","Other goods and services",regex=True)

#hack for string issue with one expenditure category?
dfExpenseToMatch = dfExpenseToMatch.replace("Personal care products and services",dfExpenseToMatch['category'][13],regex=True)
dfExpenseToMatch = dfExpenseToMatch.groupby("category").sum().reset_index()


## Import Consumer Price Index, U.S. Bureau of Labor Statistics, Western Region – May 2022
- Get inflation rate figures from monethly BLS CPI estimates for various categories and match them to the categories in the BLS Consumer Expenditure Survey.

In [36]:
tablesInflation = pd.read_html('https://www.bls.gov/regions/west/news-release/ConsumerPriceIndex_West.htm') 
dfInflation = tablesInflation[2]
#dfInflation = dfInflation['Item']['Item'].to_frame().join(dfInflation['Percent change from-']['Jul.2021'])

In [37]:
dfInflation

Item and Group  \
                                       Item and Group   
0                                Expenditure category   
1                                                 NaN   
2                                           All Items   
3                       All items (December 1977=100)   
4                                  Food and beverages   
..                                                ...   
75       Commodities less food and energy commodities   
76                                 Energy commodities   
77                      Services less energy services   
78  Footnotes (1) This index series was calculated...   
79  - Data not available Regions defined as the fo...   

                                              Indexes  \
                                       Historicaldata   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
..                                                ...   
75                                                NaN   
76                                                NaN   
77                                                NaN   
78  Footnotes (1) This index series was calculated...   
79  - Data not available Regions defined as the fo...   

                                                       \
                                              May2022   
0                                                 NaN   
1                                                 NaN   
2                                             309.645   
3                                             500.524   
4                                             312.918   
..                                                ...   
75                                            161.495   
76                                            437.032   
77                                            390.245   
78  Footnotes (1) This index series was calculated...   
79  - Data not available Regions defined as the fo...   

                                                       \
                                             Jun.2022   
0                                                 NaN   
1                                                 NaN   
2                                             313.496   
3                                             506.748   
4                                             317.106   
..                                                ...   
75                                            163.061   
76                                            471.276   
77                                            393.215   
78  Footnotes (1) This index series was calculated...   
79  - Data not available Regions defined as the fo...   

                                                       \
                                             Jul.2022   
0                                                 NaN   
1                                                 NaN   
2                                             313.951   
3                                             507.484   
4                                             319.435   
..                                                ...   
75                                            163.664   
76                                            448.036   
77                                            394.874   
78  Footnotes (1) This index series was calculated...   
79  - Data not available Regions defined as the fo...   

                                 Percent change from-  \
                                             Jul.2021   
0                                                 NaN   
1                                                 NaN   
2                                                 8.3   
3                                           

In [38]:
dfInflation = dfInflation['Item and Group']['Item and Group'].to_frame().join(dfInflation['Percent change from-']['Jul.2021'])

In [43]:
dfInflation = dfInflation.iloc[2:]

In [46]:
dfTest = dfInflation.reset_index(drop=True, inplace=True)

In [50]:
dfInflation

,Item and Group,Jul.2021
0,All Items,8.3
1,All items (December 1977=100),NaN
2,Food and beverages,10.0
3,Food,10.3
4,Food at home,12.8
...,...,...
73,Commodities less food and energy commodities,7.3
74,Energy commodities,39.8
75,Services less energy services,6.0
76,Footnotes (1) This index series was calculated...,Footnotes (1) This index series was calculated...


In [51]:
#pick out categories to match CES
dfInflationToMatch = dfInflation.loc[[3,4,11,12,14,18,23,24,25,33,34,35,37]].reset_index(drop=True)
dfInflationToMatch.columns = ['category','inflationRate']

#rename rows
dfInflationToMatch['category'].loc[5] = "Utilities, fuels, and public services"
dfInflationToMatch['category'].loc[7] = "Apparel and services"
dfInflationToMatch['category'].loc[9] = "Healthcare"
dfInflationToMatch['category'].loc[10] = "Entertainment"
dfInflationToMatch['category'].loc[11] = "Education"
dfInflationToMatch['inflationRate'] = dfInflationToMatch['inflationRate'].astype(float)
dfInflationToMatch

,category,inflationRate
0,Food,10.3
1,Food at home,12.8
2,Food away from home,6.8
3,Alcoholic beverages,4.9
4,Shelter,6.2
5,"Utilities, fuels, and public services",10.6
6,Household furnishings and operations,12.0
7,Apparel and services,5.8
8,Transportation,16.2
9,Healthcare,39.4


## Merge Inflation + Expense Data, Estimate Total Inflation Cost
- Merge inflation and expense datasets, calculate additional expenditure due to inflation for each category.

In [52]:
dfInflationExpense = pd.merge(dfInflationToMatch,dfExpenseToMatch,on="category")

In [53]:
dfInflationExpense['lowIncomeInflation'] = dfInflationExpense['lowIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense['highIncomeInflation'] = dfInflationExpense['highIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense

,category,inflationRate,lowIncomeExpense,highIncomeExpense,lowIncomeInflation,highIncomeInflation
0,Food,10.3,5578.0,11485.0,574.534,1182.955
1,Food at home,12.8,3903.0,6982.0,499.584,893.696
2,Food away from home,6.8,1675.0,4503.0,113.900,306.204
3,Alcoholic beverages,4.9,348.0,662.0,17.052,32.438
4,Shelter,6.2,9953.0,17704.0,617.086,1097.648
5,"Utilities, fuels, and public services",10.6,2762.0,4781.0,292.772,506.786
6,Household furnishings and operations,12.0,2784.0,5962.0,334.080,715.440
7,Apparel and services,5.8,963.0,2322.0,55.854,134.676
8,Transportation,16.2,4741.0,15267.0,768.042,2473.254
9,Healthcare,39.4,3445.0,6291.0,1357.330,2478.654


In [54]:
#exclude first row (Food) since next two rows are components of that.
dfIncomeExpense['inflation'] = [dfInflationExpense['lowIncomeInflation'][1:].sum().round(),dfInflationExpense['highIncomeInflation'][1:].sum().round()]
dfIncomeExpenseInflation = dfIncomeExpense

In [55]:
dfIncomeExpenseInflation

,income,expenses,inflation
lowIncome,22248,36319,5070.0
highIncome,121184,87629,11388.0


## Calculate Inflation as Portion of Income
- Take the additional inflation expenditures of each category as a proportion of an income group's average income to find the percentage of an income's groups income that has been taken up by inflationary pressure for certain categories.

In [56]:
#pick out relevant rows
dfInflationIncome = dfInflationExpense.loc[[0,8,4,5,9]]

#take proportion and round
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflation'] / dfIncomeExpense['income']['lowIncome'] * 100
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflation'] / dfIncomeExpense['income']['highIncome'] * 100
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflationProp'].round(1)
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflationProp'].round(1)

dfInflationIncome = dfInflationIncome[['category','inflationRate','lowIncomeInflationProp','highIncomeInflationProp']]
dfInflationIncome



,category,inflationRate,lowIncomeInflationProp,highIncomeInflationProp
0,Food,10.3,2.6,1.0
8,Transportation,16.2,3.5,2.0
4,Shelter,6.2,2.8,0.9
5,"Utilities, fuels, and public services",10.6,1.3,0.4
9,Healthcare,39.4,6.1,2.0


This means food inflation takes up 2.6% of the low-income households' incomes and 1% of the high-income households' incomes.
Similarly, housing inflation takes up 2.8% of the low-income households' incomes and 0.9% of the high-income households' incomes.
And so on and so on

## Get Other Inflation Rates We Care About


In [59]:
dfOtherRates = dfInflation.loc[[0,3,14,33,21,22]]
dfOtherRates = dfOtherRates.reset_index(drop=True)
dfOtherRates.columns = ['category','inflationRate']

In [60]:
dfOtherRates

,category,inflationRate
0,All Items,8.3
1,Food,10.3
2,Shelter,6.2
3,Gasoline (all types),39.4
4,Electricity(1),10.1
5,Utility (piped) gas service(1),19.7


From July 2021 to July 2022, utility prices increased 10.6% in the Western U.S. 
Electricity prices have increased 10.1% in the past year, and natural gas prices have soared 19.7%.